In [22]:
import pandas as pd
import numpy as np
import re
import time
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

import os
import sys
import urllib.request
import json
from datetime import datetime

In [23]:
### [1] URL 생성관련 함수
def file_to_df():
    df = pd.read_excel('apple_iphone_url.xlsx')
    df['inch'] = df['inch'].astype(str)
    df['germany_inch'] = df['inch'].apply(lambda x: re.sub("\.",",",x))
    df['russia_rom'] = df['rom'].apply(lambda x: re.sub("gb","",x))
    
    return df
    
def url_maker(df):
    df['us_url'] = "https://www.apple.com/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']+"-unlocked"
    df['uk_url'] = "https://www.apple.com/uk/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']
    df['china_url'] = "https://www.apple.com.cn/shop/buy-iphone/"+df['family_model']+"/"+df['china_code']+"/A"
    
    df['japan_url'] = 0
    df['germany_url'] = 0
    df['russia_url'] = 0

    for i in range(len(df)):
        df['japan_url'][i] = "https://www.apple.com/jp/shop/buy-iphone/"+df['family_model'][i]+"/"+df['inch'][i]+\
                    parse.quote(df['japan_code_inch'][i])+"-"+df['rom'][i]+"-"+parse.quote(df['japan_code_color'][i])
        df['germany_url'][i] = "https://www.apple.com/de/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"%22-display-"+\
                               df['rom'][i]+"-"+parse.quote(df['germany_code'][i]) 

        if df['family_model'][i] == "iphone-se":
            df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%BE%D0%B2%D1%8B%D0%B9-%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])
        else: 
             df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+\
                                    "%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                    df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%B0-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])       

    df['india_url'] = "https://www.indiaistore.com/"+df['model']+"/"
    
    return df

### [2]크롤링 관련 코드
def select_no(browser):
    try:
        no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
        no_answer.click()
    except: pass
    #print("click No")
          
def get_delivery_date(browser,post_code):
    #1. No 선택했는지 확인    
    browser.implicitly_wait(10)
    time.sleep(2)
    delivery_date_list = browser.find_elements_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg")
    delivery_date_li=[]

    for i in range(len(delivery_date_list)):
        delivery_date_li.append(delivery_date_list[i].text)
        #print(delivery_date_list[i])
        #print(delivery_date_li)

    delivery_date = " / ".join(delivery_date_li)
    print(delivery_date)
    return delivery_date

def get_price_delivery(browser, url, country,i,post_code):
    time.sleep(1)
    #select_no(browser)
    delivery_date = get_delivery_date(browser,post_code)
    price = browser.find_element_by_class_name('localnav-price-content').text
    price_delivery = price+"|"+delivery_date
    return price_delivery

#네이버파파고API
def tranlator_papago(text, src_lang, dest_lang):
    
    
    client_id = "Wf_OvBKP9uonTFxvBe83" 
    client_secret = "kWuIR7oLiZ" 

    data = "source=" + src_lang + "&target=" + dest_lang + "&text=" + text
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        # print(response_body.decode('utf-8'))

        #json 형 변환
        res = json.loads(response_body.decode('utf-8'))
        
    else:
        print("Error Code:" + rescode)
    return res['message']['result']['translatedText']

def select_region(browser,post_code, delivery_text):
    
    if post_code =="101000": #russia인 경우
        a = browser.find_elements_by_xpath('//*[@data-autom="deliveryDateChecker"]')[0]
        a.click()
    else: 
        get_delivery = browser.find_element_by_xpath('//button[text()="'+delivery_text+'"]')
        get_delivery.click()
        time.sleep(2)
            
            
    if post_code =="東京都":  #japan인 경우
        a= browser.find_element_by_xpath('//*[@id="state"]')
        a.click()

        b = browser.find_element_by_xpath('//option[text()="東京都"]')
        b.click()
    
    elif post_code =="100000": # china인 경우
        a= browser.find_element_by_xpath('//*[@id="state"]')
        a.click()
        time.sleep(0.5)
        b = browser.find_element_by_xpath('//option[text()="北京"]')
        b.click()
        
        time.sleep(1)
        a= browser.find_element_by_xpath('//*[@id="city"]')
        a.click()
        time.sleep(1)
        b = browser.find_element_by_xpath('//*[@id="city"]/option[2]')
        b.click()
        
        time.sleep(1)
        a= browser.find_element_by_xpath('//*[@id="district"]')
        a.click()
        time.sleep(1)
        b = browser.find_element_by_xpath('//option[text()="东城区"]')
        b.click()
        
        try:
            input_region_code = browser.find_elements_by_xpath('//*[@id="postalCode"]')[0]
            input_region_code.send_keys(post_code)

        except:
            input_region_code = browser.find_elements_by_xpath('//*[@id="postalCode"]')
            browser.execute_script("arguments[0].click();", input_region_code)
            input_region_code.send_keys(post_code)
        
        
    else: 
        try:
            input_region_code = browser.find_elements_by_xpath('//*[@id="postalCode"]')[0]
            input_region_code.send_keys(post_code)

        except:
            time.sleep(2)
            input_region_code = browser.find_elements_by_xpath('//*[@id="postalCode"]')[0]
            input_region_code.send_keys(post_code)
#             input_region_code = browser.find_elements_by_xpath('//*[@id="postalCode"]')
#             browser.execute_script("arguments[0].click();", input_region_code)
#             input_region_code.send_keys(post_code)

    ok_button = browser.find_element_by_class_name("as-deliverydatesoverlay-geospan")
    ok_button.click()

    webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform()        
        
    print("     > 우편번호 설정 완료*******")
    
def select_no(browser):
    try:
        no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
        no_answer.click()
        #print("click No")
    except:
        no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
        browser.execute_script("arguments[0].click();", no_answer)
    

In [24]:
# <1> 기본 DataFrame을 세팅한다.
#--> 실행할때마다 df가 초기화되므로 다른 cell로 뺌!!!!!!!
df = file_to_df()
df = url_maker(df)
browser = webdriver.Chrome()

In [25]:
# browser = webdriver.Chrome()

In [30]:
# <2> 이제부터 포문돌리면서 수집 시작!
url_list =[('usa','us_url','10001','Get delivery dates'),                               #0   usa
           ('uk','uk_url','sw1a2aa','Get delivery dates'),                              #1   uk
           ('germany','germany_url','10115','Liefertermine abrufen'),                   #2   germany/de
           ('russia','russia_url','101000','Узнайте дату доставки'), #3   russia/ru
           ('china', 'china_url','100000','查看送货日期'),                              #4   china/cn
           ('japan', 'japan_url','東京都','さらに配送オプションを表示'),                #5   japan
           ('india', 'india_url')]                                                      #6   india

# for con_num in range(0,1):
for con_num in range(1,6):
    country = url_list[con_num][0]        # 국가 이름
    res_col = url_list[con_num][0]+"_res" # 컬럼명1
    col_nm = url_list[con_num][1]         # url
    post_code = url_list[con_num][2]      # 지정 우편번호
    delivery_text = url_list[con_num][3]
    url = df[col_nm][0]                   #---> us_url, uk_url, china_url, japan_url, germany_url, russia_url, india_url 
    df[res_col] = 0
    print("     > "+str(con_num)+"번 국가: "+country+" 시작합니다.")
    
    
#     for i in range(44,63):
    for i in range(len(df['model'])):
        url = df[col_nm][i]
        browser.get(url)

        if i==0: #처음이면 지역선택   
            if con_num!=3: #러시아면 No 선택할필요 없음
                select_no(browser)
                
            else: pass
            
            time.sleep(4)
            select_region(browser,post_code, delivery_text)
            time.sleep(2) 
        
        elif i in [30,54]: 
            if con_num!=3: #러시아면 No 선택할필요 없음
                select_no(browser)
                
            else: pass
        else: pass
        
        print(i, end = " ")    
        price_delivery_list = get_price_delivery(browser, url,country,i,post_code)
        df[res_col][i] = price_delivery_list

    #display(df.head(2))

     > 1번 국가: uk 시작합니다.
     > 우편번호 설정 완료*******
0 Mon 15 Feb – Free
1 Mon 15 Feb – Free
2 Mon 15 Feb – Free
3 Mon 15 Feb – Free
4 Mon 15 Feb – Free
5 Mon 15 Feb – Free
6 Mon 15 Feb – Free
7 Mon 15 Feb – Free
8 Mon 15 Feb – Free
9 Mon 15 Feb – Free
10 Mon 15 Feb – Free
11 Mon 15 Feb – Free
12 Mon 15 Feb – Free
13 Mon 15 Feb – Free
14 Mon 15 Feb – Free
15 Mon 15 Feb – Free
16 Mon 15 Feb – Free
17 Mon 15 Feb – Free
18 Mon 15 Feb – Free
19 Mon 15 Feb – Free
20 Mon 15 Feb – Free
21 Mon 15 Feb – Free
22 Mon 15 Feb – Free
23 Mon 15 Feb – Free
24 Mon 15 Feb – Free
25 Mon 15 Feb – Free
26 Mon 15 Feb – Free
27 Mon 15 Feb – Free
28 Mon 15 Feb – Free
29 Mon 15 Feb – Free
30 18 Feb - 22 Feb – Free
31 18 Feb - 22 Feb – Free
32 18 Feb - 22 Feb – Free
33 18 Feb - 22 Feb – Free
34 18 Feb - 22 Feb – Free
35 18 Feb - 22 Feb – Free
36 Tue 16 Feb – Free
37 18 Feb - 22 Feb – Free
38 Tue 16 Feb – Free
39 18 Feb - 22 Feb – Free
40 18 Feb - 22 Feb – Free
41 18 Feb - 22 Feb – Free
42 Mon 15 Feb – Free
43 Mon 1

22 土 2021/02/13 – 送料無料
23 土 2021/02/13 – 送料無料
24 土 2021/02/13 – 送料無料
25 土 2021/02/13 – 送料無料
26 土 2021/02/13 – 送料無料
27 土 2021/02/13 – 送料無料
28 土 2021/02/13 – 送料無料
29 土 2021/02/13 – 送料無料
30 土 2021/02/13 – 送料無料
31 土 2021/02/13 – 送料無料
32 土 2021/02/13 – 送料無料
33 土 2021/02/13 – 送料無料
34 土 2021/02/13 – 送料無料
35 土 2021/02/13 – 送料無料
36 土 2021/02/13 – 送料無料
37 土 2021/02/13 – 送料無料
38 土 2021/02/13 – 送料無料
39 土 2021/02/13 – 送料無料
40 土 2021/02/13 – 送料無料
41 土 2021/02/13 – 送料無料
42 土 2021/02/13 – 送料無料
43 土 2021/02/13 – 送料無料
44 土 2021/02/13 – 送料無料
45 土 2021/02/13 – 送料無料
46 土 2021/02/13 – 送料無料
47 土 2021/02/13 – 送料無料
48 土 2021/02/13 – 送料無料
49 土 2021/02/13 – 送料無料
50 土 2021/02/13 – 送料無料
51 土 2021/02/13 – 送料無料
52 土 2021/02/13 – 送料無料
53 土 2021/02/13 – 送料無料
54 土 2021/02/13 – 送料無料
55 土 2021/02/13 – 送料無料
56 土 2021/02/13 – 送料無料
57 土 2021/02/13 – 送料無料
58 土 2021/02/13 – 送料無料
59 土 2021/02/13 – 送料無料
60 土 2021/02/13 – 送料無料
61 土 2021/02/13 – 送料無料
62 土 2021/02/13 – 送料無料


In [31]:
today_date=datetime.today().strftime("%Y%m%d")
df.to_excel('iphone_New_before_trans_'+today_date+'.xlsx')
df = pd.read_excel('iphone_New_before_trans_'+today_date+'.xlsx')
# df = pd.read_excel('iphone_New_before_trans_20210118.xlsx')

In [32]:
# <1> result 결과만 가져와서 새로운 DataFrame 생성
df_b = df[['usa_res', 'uk_res','china_res','japan_res','germany_res','russia_res']]
df_b['china_res'].value_counts()[:2]

RMB 283/月或 RMB 6799|星期日 2021/02/21 – 免费    10
RMB 229/月或 RMB 5499|星期日 2021/02/21 – 免费     5
Name: china_res, dtype: int64

In [33]:
# <2> "|"를 기준으로 가격과 Delivery Date 구분

df_b['usa_res'] = df_b['usa_res'].apply(lambda x: "mo. or $00.00|수집안됨 " if "|" not in str(x) else x)
df_b['us_price'] = df_b['usa_res'].apply(lambda x : str(x).split("or ")[1].split("|")[0])
df_b['us_delivery'] = df_b['usa_res'].apply(lambda x : x.split("or ")[1].split("|")[1])

df_b['uk_price'] = df_b['uk_res'].apply(lambda x : x.split(" or ")[1].replace("**","").split("|")[0])
df_b['uk_delivery'] = df_b['uk_res'].apply(lambda x : x.split(" or ")[1].replace("**","").split("|")[1])

df_b['ch_price'] = df_b['china_res'].apply(lambda x : x.replace("或\n","月或 ").split("月或 ")[1].split("|")[0])
df_b['ch_delivery'] = df_b['china_res'].apply(lambda x : x.replace("或\n","月或 ").split("月或 ")[1].split("|")[1])

df_b['ja_price'] = df_b['japan_res'].apply(lambda x : x.replace("（税別）","").split("|")[0])
df_b['ja_delivery'] = df_b['japan_res'].apply(lambda x : x.replace("（税別）","").split("|")[1])

df_b['de_price'] = df_b['germany_res'].apply(lambda x : x.split(" oder ")[1].replace("**","").split("|")[0])
df_b['de_delivery'] = df_b['germany_res'].apply(lambda x : x.split(" oder ")[1].replace("**","").split("|")[1])

df_b['ru_price'] = df_b['russia_res'].apply(lambda x : x.split("|")[0])
df_b['ru_delivery'] = df_b['russia_res'].apply(lambda x : x.split("|")[1])

df_b[['ch_del_tr', 'ja_del_tr' , 'de_del_tr', 'ru_del_tr']]=0

for i in range(len(df_b)):
    print(i, end="\r")
    chinese_word = df_b['ch_delivery'][i]
    if chinese_word=="": pass
    else:
        chinese_translated = tranlator_papago(chinese_word, 'zh-CN', 'en')
        df_b['ch_del_tr'][i] = chinese_translated
    
    japanese_word = df_b['ja_delivery'][i]
    if japanese_word=="": pass
    else: 
        japanese_translated = tranlator_papago(japanese_word, 'ja', 'en')
        df_b['ja_del_tr'][i] = japanese_translated 
    
    try:
        germany_word = df_b['de_delivery'][i]
        if germany_word=="": pass
        else: 
            germany_translated =  tranlator_papago(germany_word, 'de', 'ko')
            df_b['de_del_tr'][i] = germany_translated
    except: df_b['de_del_tr'][i]=0
    #print(chinese_translated,'\n', japanese_translated,'\n', germany_translated,'\n', russia_translated)

In [34]:
df_b.to_excel("translated2_"+today_date+".xlsx")
#df_b.to_excel("translated2_20210118.xlsx")
df_b[['ch_del_tr','ja_del_tr','de_del_tr','ru_del_tr']][:10]

,ch_del_tr,ja_del_tr,de_del_tr,ru_del_tr
0,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
1,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
2,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
3,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
4,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
5,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
6,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
7,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
8,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
9,Sunday 2021/02/21 – free,Saturday 2021/02/13 – Free shipping,"2월 13일 - 10,00 € / 2월 15일 무료",0
